In [3]:
# Importing Dependencies and Setup
# We don't know what we need, so we'll import a bunch of different modules
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import time
from scipy.stats import linregress

In [4]:
# Load CSV file and turn it into PD DataFrames
goOne_df = pd.read_csv("/Users/mdcummings/Desktop/Data Analysis/Boot Camp/Project 1/goemotions_1.csv")
goTwo_df = pd.read_csv("/Users/mdcummings/Desktop/Data Analysis/Boot Camp/Project 1/goemotions_2.csv")
goThree_df = pd.read_csv("/Users/mdcummings/Desktop/Data Analysis/Boot Camp/Project 1/goemotions_3.csv")

In [5]:
# Merge the three separate DataFrames into one.
dfMerge = goOne_df.merge(goTwo_df, how='outer')
dfMerge = dfMerge.merge(goThree_df, how='outer')

# Convert column into DateTime datatype, and apply the correct unit (seconds) to turn it readable.
dfMerge['created_utc'] = dfMerge['created_utc'].astype('datetime64[s]')
dfMerge

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,eew5j0j,Brdd9,nrl,t3_ajis4z,t1_eew18eq,2019-01-25 01:50:39,1,False,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,eemcysk,TheGreen888,unpopularopinion,t3_ai4q37,t3_ai4q37,2019-01-21 15:22:49,37,True,0,...,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",ed2mah1,Labalool,confessions,t3_abru74,t1_ed2m7g7,2019-01-02 11:15:44,37,False,0,...,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,eeibobj,MrsRobertshaw,facepalm,t3_ahulml,t3_ahulml,2019-01-20 06:17:34,18,False,0,...,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",eda6yn6,American_Fascist713,starwarsspeculation,t3_ackt2f,t1_eda65q2,2019-01-05 06:10:01,2,False,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211220,Everyone likes [NAME].,ee6pagw,Senshado,heroesofthestorm,t3_agjf24,t3_agjf24,2019-01-16 10:25:22,16,False,0,...,1,0,0,0,0,0,0,0,0,0
211221,Well when you’ve imported about a gazillion of...,ef28nod,5inchloser,nottheonion,t3_ak26t3,t3_ak26t3,2019-01-27 01:39:42,15,False,0,...,0,0,0,0,0,0,0,0,0,0
211222,That looks amazing,ee8hse1,springt1me,shittyfoodporn,t3_agrnqb,t3_agrnqb,2019-01-17 00:08:54,70,False,1,...,0,0,0,0,0,0,0,0,0,0
211223,The FDA has plenty to criticize. But like here...,edrhoxh,enamedata,medicine,t3_aejqzd,t1_edrgdtx,2019-01-11 01:07:12,4,False,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Clean DataFrame by: 1.# Dropping columns, 2.) Choosing "Clear" selections, 3.) Dropping the "example_very_unclear" column after selection
dfStaging = dfMerge.drop(columns=['id', 'author', 'link_id', 'parent_id'])
dfStagingA = dfStaging.loc[(dfStaging['example_very_unclear'] == False)]
dfClean = dfStagingA.drop(columns=['example_very_unclear'])
dfClean
# dfClean will have "clear" examples

,text,subreddit,created_utc,rater_id,admiration,amusement,anger,annoyance,approval,caring,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,nrl,2019-01-25 01:50:39,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,"You do right, if you don't care then fuck 'em!",confessions,2019-01-02 11:15:44,37,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,facepalm,2019-01-20 06:17:34,18,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",starwarsspeculation,2019-01-05 06:10:01,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,Right? Considering it’s such an important docu...,TrueReddit,2019-01-23 21:50:08,61,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211219,"Well, I'm glad you're out of all that now. How...",raisedbynarcissists,2019-01-04 15:22:34,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
211220,Everyone likes [NAME].,heroesofthestorm,2019-01-16 10:25:22,16,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
211221,Well when you’ve imported about a gazillion of...,nottheonion,2019-01-27 01:39:42,15,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
211222,That looks amazing,shittyfoodporn,2019-01-17 00:08:54,70,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# Clean DataFrame by: 1.# Dropping columns, 2.) Choosing "Unclear" selections, 3.) Dropping the "example_very_unclear" column after selection
# Same as above; 
dfStagingB = dfStaging.loc[(dfStaging['example_very_unclear'] == True)]
dfEVUClean = dfStagingB.drop(columns=['example_very_unclear'])
dfEVUClean
# dfEVUClean will have "unclear" examples

,text,subreddit,created_utc,rater_id,admiration,amusement,anger,annoyance,approval,caring,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
1,>sexuality shouldn’t be a grouping category I...,unpopularopinion,2019-01-21 15:22:49,37,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62,Oi us cancers don’t want him either! We respec...,Justfuckmyshitup,2019-01-13 22:13:56,22,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
195,I unfortunately can not afford a lawyer. Lucki...,Divorce,2019-01-01 17:28:15,12,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
262,OMG THOSE TINY SHOES! *desire to boop snoot in...,chicago,2019-01-20 05:36:33,61,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
264,I would pay money for [NAME] to not be my nurs...,SoulCalibur,2019-01-17 03:50:29,22,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211037,_____ is ruining _____ because kids these days.,moviescirclejerk,2019-01-10 23:51:59,57,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
211082,"yea I agree, and [NAME] is way younger so has ...",Dodgers,2019-01-04 22:09:22,37,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
211113,"Huh, assumed he was more of a grindr fella .....",90dayfianceuncensored,2019-01-30 23:05:53,61,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
211214,Well when you’ve imported about a gazillion of...,nottheonion,2019-01-27 01:39:42,61,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
dfEVUTrue = dfMerge.loc[dfMerge['example_very_unclear'] == True]

In [9]:
subS = goOne_df.pivot_table(index = ['subreddit'], aggfunc ='size')
subS.sort_values(inplace=True)
subS

subreddit
farcry               26
canes                37
Anticonsumption      39
OttawaSenators       44
TeamSolomid          48
                   ... 
AnimalsBeingBros    277
loveafterlockup     281
timberwolves        292
socialanxiety       303
cringe              319
Length: 483, dtype: int64

In [10]:
sub_G1 = goOne_df.loc[(goOne_df['subreddit'] == 'AnimalsBeingBros') | (goOne_df['subreddit'] == 'cringe') | (goOne_df['subreddit'] == 'socialanxiety') & (goOne_df['example_very_unclear'] == True), :]
sub_G1.drop(columns=['author', 'id', 'link_id', 'parent_id', 'created_utc', 'example_very_unclear'])

,text,subreddit,rater_id,admiration,amusement,anger,annoyance,approval,caring,confusion,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
6,"He isn't as big, but he's still quite popular....",cringe,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31,"Something Something Something, space of aids",cringe,19,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
59,Gotta protect’em!,AnimalsBeingBros,34,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
383,NO MAN LEFT BEHIND,AnimalsBeingBros,55,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
557,Yeah it doesn't impact much except when they b...,cringe,56,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69307,Oh boy... it must actually be kind of nice to ...,cringe,66,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
69495,Wonder where all the shit is.,AnimalsBeingBros,67,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
69602,Oh cool another manufactured awful [NAME] push...,cringe,46,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69669,"""one of us will be right!"" Or...ya both wrong ...",cringe,39,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
sub_ABB = sub_G1.loc[(sub_G1['subreddit'] == 'AnimalsBeingBros')]
sub_ABB.drop(columns=['author', 'id', 'link_id', 'parent_id', 'created_utc', 'example_very_unclear', 'rater_id'], inplace=True)
sub_ABB.reset_index(inplace=True, drop=True)
ABBad = sub_ABB.admiration()
ABB = len(sub_ABB.index)
print(f"AnimalsBeingBros has {ABB} comments and {ABBad} admirations")

/Users/mdcummings/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


TypeError: 'Series' object is not callable

In [ ]:
sub_ABB

,text,subreddit,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,Gotta protect’em!,AnimalsBeingBros,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NO MAN LEFT BEHIND,AnimalsBeingBros,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,It’s possible that the dog just escaped. Espec...,AnimalsBeingBros,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,They r playing,AnimalsBeingBros,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,I recently learned that cats lick to show domi...,AnimalsBeingBros,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,That dog actually looks flatter too.,AnimalsBeingBros,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
273,Just being friendly and sociable for the famil...,AnimalsBeingBros,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
274,"That's crazy, my brother's cat did the same th...",AnimalsBeingBros,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
275,Technology is amazing. Now we can SEE how new ...,AnimalsBeingBros,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
sub_Cri = sub_G1.loc[(sub_G1['subreddit'] == 'cringe')]
sub_Cri.drop(columns=['author', 'id', 'link_id', 'parent_id', 'created_utc', 'example_very_unclear', 'rater_id']).reset_index(inplace=True, drop=True)
sub_ABB

,text,subreddit,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,Gotta protect’em!,AnimalsBeingBros,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NO MAN LEFT BEHIND,AnimalsBeingBros,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,It’s possible that the dog just escaped. Espec...,AnimalsBeingBros,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,They r playing,AnimalsBeingBros,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,I recently learned that cats lick to show domi...,AnimalsBeingBros,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,That dog actually looks flatter too.,AnimalsBeingBros,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
273,Just being friendly and sociable for the famil...,AnimalsBeingBros,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
274,"That's crazy, my brother's cat did the same th...",AnimalsBeingBros,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
275,Technology is amazing. Now we can SEE how new ...,AnimalsBeingBros,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
sub_ABB.drop(columns=['text', 'subreddit'], inplace=True)

/Users/mdcummings/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
sub_ABB

NameError: name 'sub_ABB' is not defined

In [12]:
# applying Numpy.select to the clean dataframe which will scan selected columns based on conditions and output the sentiments for each emotion selected
# dfClean
emo_conditions = [
    (dfClean['amusement'] == 1),
    (dfClean['excitement'] == 1),
    (dfClean['joy'] == 1),
    (dfClean['love'] == 1),
    (dfClean['desire'] == 1),
    (dfClean['optimism'] == 1),
    (dfClean['caring'] == 1),
    (dfClean['pride'] == 1),
    (dfClean['admiration'] == 1),
    (dfClean['gratitude'] == 1),
    (dfClean['relief'] == 1),
    (dfClean['approval'] == 1),
    (dfClean['realization'] == 1),
    (dfClean['surprise'] == 1),
    (dfClean['curiosity'] == 1),
    (dfClean['confusion'] == 1),
    (dfClean['fear'] == 1),
    (dfClean['nervousness'] == 1),
    (dfClean['remorse'] == 1),
    (dfClean['embarrassment'] == 1),
    (dfClean['disappointment'] == 1),
    (dfClean['sadness'] == 1),
    (dfClean['grief'] == 1),
    (dfClean['disgust'] == 1),
    (dfClean['anger'] == 1),
    (dfClean['annoyance'] == 1),
    (dfClean['disapproval'] == 1)
]

sentiments = ['Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
              'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
               'Ambiguous', 'Ambiguous','Ambiguous','Ambiguous',
               'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative',
               'Negative', 'Negative', 'Negative', 'Negative',]

dfClean['Sentiment'] = np.select(emo_conditions, sentiments)
dfClean.head(50)

,text,subreddit,created_utc,rater_id,admiration,amusement,anger,annoyance,approval,caring,...,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral,Sentiment
0,That game hurt.,nrl,2019-01-25 01:50:39,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Negative
2,"You do right, if you don't care then fuck 'em!",confessions,2019-01-02 11:15:44,37,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,Man I love reddit.,facepalm,2019-01-20 06:17:34,18,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Positive
4,"[NAME] was nowhere near them, he was by the Fa...",starwarsspeculation,2019-01-05 06:10:01,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,Right? Considering it’s such an important docu...,TrueReddit,2019-01-23 21:50:08,61,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Positive
6,"He isn't as big, but he's still quite popular....",cringe,2019-01-01 05:21:16,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Negative
7,That's crazy; I went to a super [RELIGION] hig...,TeenMomOGandTeenMom2,2019-01-03 17:22:38,23,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Positive
8,that's adorable asf,traaaaaaannnnnnnnnns,2019-01-29 12:05:43,73,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Positive
9,"""Sponge Blurb Pubs Quaw Haha GURR ha AAa!"" fin...",youtubehaiku,2019-01-08 21:38:34,54,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Positive
10,"I have, and now that you mention it, I think t...",AskMenOver30,2019-01-05 03:17:43,36,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
